# Build a model with the data

In [11]:
import numpy as np
import pandas as pd
import sqlite3
import pickle

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.stem.porter import PorterStemmer


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import classification_report, accuracy_score, f1_score

In [12]:
conn = sqlite3.connect('../data/disaster_tweets.db')
df = pd.read_sql('SELECT * FROM tweets', con = conn)

In [63]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'PII', 'request',
       'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [71]:
X = df['message']
Y = df[['related','request',
       'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report']]

category_names = list(Y.columns)

Here are disaster related messages.

In [72]:
for i in range(10):
    print(X[Y['related'] == 1].iloc[i] + '\n')

Weather update - a cold front from Cuba that could pass over Haiti

Is the Hurricane over or is it not over

says: west side of Haiti, rest of the country today and tonight

Storm at sacred heart of jesus

Please, we need tents and water. We are in Silo, Thank you!

There's nothing to eat and water, we starving and thirsty.

I am in Thomassin number 32, in the area named Pyron. I would like to have some water. Thank God we are fine, but we desperately need water. Thanks

Let's do it together, need food in Delma 75, in didine area

More information on the 4636 number in order for me to participate. ( To see if I can use it )

A Comitee in Delmas 19, Rue ( street ) Janvier, Impasse Charite #2. We have about 500 people in a temporary shelter and we are in dire need of Water, Food, Medications, Tents and Clothes. Please stop by and see us.



Here are non-disaster related messages.

In [73]:
for i in range(10):
    print(X[Y['related'] == 0].iloc[i] + '\n')

Information about the National Palace-

I would like to receive the messages, thank you

I am in Petionville. I need more information regarding 4636

I don't understand how to use this thing 4636.

Can you tell me about this service

Good evening, Radio one please. I would like information on Tiyous.

I'm here, I didn't find the person that I needed to send the pant by phone

I'm listening to you at Miraguan we asking the government to take change because one gallon gas is 80.

i am very happy, i hear god, religious hyme

I would like to know how food is distributed.



In [74]:
porter = PorterStemmer()
def tokenize(text):
    return [porter.stem(word) for word in text.split()]

In [90]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LogisticRegression(class_weight='balanced',multi_class='ovr')))
])

In [91]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [92]:
pipeline.fit(X_train, Y_train)

/Users/andrewblaikie/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=LogisticRegression(C=1.0,
                                                                    class_weight='balanced',
           

In [95]:
Y_pred = pipeline.predict(X_test)

In [96]:
# Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print('Category: {} '.format(category_names[i]))
    print('f1 Score: {} '.format(f1_score(Y_test.iloc[:, i].values, Y_pred[:, i])))
    print('Accuracy {}\n\n'.format(accuracy_score(Y_test.iloc[:, i].values, Y_pred[:, i])))

Category: related 
f1 Score: 0.8609711651838353 
Accuracy 0.8008903899293829


Category: request 
f1 Score: 0.6702168124762267 
Accuracy 0.8669020571077679


Category: offer 
f1 Score: 0.05042016806722689 
Accuracy 0.9826527479275406


Category: aid_related 
f1 Score: 0.7396333867236163 
Accuracy 0.7754068160884249


Category: medical_help 
f1 Score: 0.422972972972973 
Accuracy 0.8688977586736261


Category: medical_products 
f1 Score: 0.39840637450199207 
Accuracy 0.907276634940129


Category: search_and_rescue 
f1 Score: 0.2473867595818815 
Accuracy 0.933681301811483


Category: security 
f1 Score: 0.11176470588235295 
Accuracy 0.9536383174700644


Category: military 
f1 Score: 0.45973154362416113 
Accuracy 0.9505680073687442


Category: water 
f1 Score: 0.6482504604051565 
Accuracy 0.9413570770647836


Category: food 
f1 Score: 0.7079326923076923 
Accuracy 0.9253914645379183


Category: shelter 
f1 Score: 0.5990271021542738 
Accuracy 0.9114215535769112


Category: clothing 
f1 Score

In [97]:
pickle.dump(pipeline, open('trained_classifier.pkl', 'wb'))

In [98]:
# Load the model from pickle file

model = pickle.load(open('trained_classifier.pkl', 'rb'))

In [99]:
def get_predicted_category_names(category_predicted):
    return [category_names[i] for i in range(len(category_predicted)) if category_predicted[i] == 1]

In [101]:
get_predicted_category_names(model.predict(["I felt a big storm in Eugene this morning. I need water."])[0])

['related', 'request', 'water', 'weather_related', 'storm', 'direct_report']

In [102]:
get_predicted_category_names(model.predict(["How do I download zoom for this remote meeting?"])[0])

[]